# Problem Statement
## Exercise 2.12 from Operations Research: Models and Methods by Jensen & Bard

Ten jobs are to be completed by three workers during the next week. Each worker has a 40-hour work week. The times for the workers to complete the jobs are shown in the table. The values in the cells assume that each job is completed by a single worker; however, jobs can be shared, with completion times being determined proportionally If no entry exists in a particular cell, it means that the corresponding job cannot be performed by the corresponding worker. Set up and solve an LP model that will determine the optimal assignment of workers to jobs. The goal is to minimize the total time required to complete all the jobs.




In [1]:
import pandas as pd
import pyomo.environ as pe
import pyomo.opt as po


workers = {'A', 'B', 'C'}

tasks = set(range(1, 11))

c = {
    ('A',  2):  7,
    ('A',  3):  3,
    ('A',  6): 18,
    ('A',  7): 13,
    ('A',  8):  6,
    ('A', 10):  9,
    ('B',  1): 12,
    ('B',  2):  5,
    ('B',  4): 12,
    ('B',  5):  4,
    ('B',  6): 22,
    ('B',  8): 17,
    ('B',  9): 13,
    ('C',  1): 18,
    ('C',  3):  6,
    ('C',  4):  8,
    ('C',  5): 10,
    ('C',  7): 19,
    ('C',  9):  8,
    ('C', 10): 15,
}

max_hours = 40


Define $W$ as the set of workers and $T$ as the sets of tasks. Also, define $c_{w t}$ as the number of hours worker $w$ requires to complete task $t$. (Note that we do not explicitly prohibit a worker from completiting as task; rather, we make the cost arbitrarily large if worker $w$ is unable to perform task $t$.) Let $x_{w t}$ be the proportion of task $t$ that is completed by worker $j$. Let $H$ be the max number of hours that any single worker may log in a week. We formulate as follows.
$$
\begin{array}{lll}
\operatorname{minimize} & \sum_{w \in W} \sum_{t \in T} c_{w t} x_{w t} & \\
\text { subject to } & \sum_{t \in T} c_{w t} x_{w t} \leq H, \quad \forall w \in W \\
& \sum_{w \in W} x_{w t}=1 & \forall t \in T \\
& 0 \leq x_{w t} \leq 1, & \forall w \in W, \forall t \in T
\end{array}
$$

# Implementacion
## Modelo

In [2]:
model = pe.ConcreteModel()

## Conjuntos

In [3]:
model.workers = pe.Set(initialize=workers)
model.tasks = pe.Set(initialize=tasks)

    source (type: set).  This WILL potentially lead to nondeterministic
    behavior in Pyomo
    source (type: set).  This WILL potentially lead to nondeterministic
    behavior in Pyomo


## Parametros

In [4]:
model.c = pe.Param(model.workers, model.tasks, initialize=c, default=1000)
model.max_hours = pe.Param(initialize=max_hours)

In [5]:
## Variables
model.x = pe.Var(model.workers, model.tasks, domain=pe.Reals, bounds=(0, 1))


In [6]:
## Funcion Objetivo
expr = sum(model.c[w, t] * model.x[w, t]           for w in model.workers for t in model.tasks)
model.objective = pe.Objective(sense=pe.minimize, expr=expr)

In [7]:
## Restricciones
### Que se cumplent todas las tareas
model.tasks_done = pe.ConstraintList()
for t in model.tasks:
    lhs = sum(model.x[w, t] for w in model.workers)
    rhs = 1
    model.tasks_done.add(lhs == rhs)
### Que se cumple el maximo de horas por trabajdor
model.hour_limit = pe.ConstraintList()
for w in model.workers:
    lhs = sum(model.c[w, t] * model.x[w, t] for t in model.tasks)
    rhs = model.max_hours
    model.hour_limit.add(lhs <= rhs)

In [8]:
solver = po.SolverFactory('gurobi')
results = solver.solve(model, tee=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2022-08-06
Read LP format model from file C:\Users\juano\AppData\Local\Temp\tmpvvn5ou1t.pyomo.lp
Reading time = 0.00 seconds
x31: 14 rows, 31 columns, 61 nonzeros
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads
Optimize a model with 14 rows, 31 columns and 61 nonzeros
Model fingerprint: 0xb4795d12
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [3e+00, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve removed 1 rows and 1 columns
Presolve time: 0.02s
Presolved: 13 rows, 30 columns, 60 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.8000000e+01   1.200000e+01   0.000000e+00      0s
      10    8.8000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.03 seconds (0.00 work units)
Optimal objective  8

In [8]:
sum(model.c[w, t] * model.x[w, t]           for w in model.workers for t in model.tasks)

In [11]:
model.x